In [3]:
import pandas as pd
df=pd.read_excel("inter-rater.xlsx")
new_keys=['embedUrl',
 'videoId',
 'title',
 'isIncluded.niu',
 'diagnostic.niu',
 'mention.niu',
 'prognostic.niu',
 'motivational_action.niu',
 'motivational_rationale.niu',
 'isIncluded.cat',
 'diagnostic.cat',
 'mention.cat',
 'prognostic.cat',
 'motivational_action.cat',
 'motivational_rationale.cat',
 'isIncluded.kathy',
 'diagnostic.kathy',
 'mention.kathy',
 'prognostic.kathy',
 'motivational_action.kathy',
 'motivational_rationale.kathy']
df.columns=new_keys
new_keys=['embedUrl',
 'videoId',
 'title',
 'isIncluded.niu',
 'isIncluded.cat',
 'isIncluded.kathy',
 'diagnostic.niu',
 'diagnostic.cat',
 'diagnostic.kathy',
 'mention.niu',
 'mention.cat',
 'mention.kathy',
 'prognostic.niu',
 'prognostic.cat',
 'prognostic.kathy',
 'motivational_action.niu',
 'motivational_action.cat',
 'motivational_action.kathy',
 'motivational_rationale.niu',
 'motivational_rationale.cat',
 'motivational_rationale.kathy']
df=df[new_keys]
df.to_excel("inter_rater2.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'inter-rater.xlsx'

In [1]:
import nltk
from nltk.metrics import agreement,masi_distance,jaccard_distance,binary_distance
from nltk.metrics.agreement import AnnotationTask
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
import numpy as np

mlb = MultiLabelBinarizer()
arr = mlb.fit_transform([{'none_apply'}, {'climate','forest'} , {'fr'}])
list(mlb.classes_)

class color:
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# cosine similarity is assuming full rank matrices, won't work for rank-deficient
# go through dimension reduction for sparse matrix
def cosine_distance(vec_a,vec_b):
#     vec_a, vec_b = list(vec_a), list(vec_b)
#     print(type(vec_a))
    distance = spatial.distance.cosine(vec_a,vec_b)
    return distance

# dropping columns with all 0s in the matrix
def dim_reduction(vector):
    vector = np.array(vector)
    idx_zero_cols = np.argwhere(np.all(vector[..., :] == 0, axis=0))
    res = np.delete(vector, idx_zero_cols, axis=1)
    return res

def toList(object):
    taglist = object[1:-1].split(";")
    taglist = ["video_not_included" if tag=="" else tag for tag in taglist]
    return taglist

def calculate_score_cosine(question, task_data):
    distance_fn = binary_distance if question=="mention" else cosine_distance
    cosine_task = AnnotationTask(data=task_data,distance = distance_fn)
    print(f"Fleiss's Kappa using Cosine distance: {cosine_task.multi_kappa()}")
#     print(f"Krippendorff's Alpha using Cosine distance: {cosine_task.alpha()}")

def calculate_score_jaccard(question, task_data):
    distance_fn = binary_distance if question=="mention" else jaccard_distance
    jaccard_task = AnnotationTask(data=task_data,distance = distance_fn)
    print(f"Fleiss's Kappa using Jaccard distance: {jaccard_task.multi_kappa()}")
#     print(f"Krippendorff's Alpha using Jaccard distance: {jaccard_task.alpha()}")

    
# for each row in df_cat:
# take other rows in dfniu and dfkathy, turn into dict for each row of each df
# combine them into a list -> go through mlb_fit
# loop through array and create annotation set of (coder, video, array_row) -> add to task_data 

def generate_task_data_cosine(question, coder_dict):
    mlb = MultiLabelBinarizer()
    if not coder_dict:
        return
    df_seed = list(coder_dict.values())[0]
    task_data = []
    for index, row in df_seed.iterrows():
        answer_list = []
        for coder, df_coder in coder_dict.items():
            answer_list.append(set(toList(df_coder.iloc[index][question])))
        print(answer_list)
        arr = dim_reduction(mlb.fit_transform(answer_list))
        for pair in zip([list(r) for r in arr],coder_dict.keys()):
            answer, coder = pair
            annotation = coder, row["videoId"], tuple(answer)
            task_data.append(annotation)
    return task_data

def generate_task_data_jaccard(question, coder_dict):
    task_data = []
    for coder, df_coder in coder_dict.items():
        for index, row in df_coder.iterrows():
            annotation = toAnnotation(coder,row,question)
            task_data.append(annotation)
    return task_data

def toFrozenSet(object):
    taglist = object[1:-1].split(";")
    taglist = ["video_not_included" if tag=="" else tag for tag in taglist]
    return frozenset(taglist)

def toAnnotation(coder, row, question):
    item = row["videoId"]
    tagging = toFrozenSet(row[question])
    return coder, item, tagging
        

In [24]:
df=pd.read_excel("inter_rater2.xlsx")
QUESTION = ["diagnostic",
             "mention",
             "prognostic",
             "motivational_action",
             "motivational_rationale"]

cat_cols=["videoId"]+[q+".cat" for q in QUESTION]
niu_cols=["videoId"]+[q+".niu" for q in QUESTION]
kathy_cols=["videoId"]+[q+".kathy" for q in QUESTION]

df_cat=df[cat_cols]
df_cat.columns=["videoId"]+QUESTION
df_niu=df[niu_cols]
df_niu.columns=["videoId"]+QUESTION
df_kathy=df[kathy_cols]
df_kathy.columns=["videoId"]+QUESTION

display(df_cat)
CODER_DICT = {"cat":df_cat, "niu":df_niu, "kathy":df_kathy}

for question in QUESTION:
    print(color.BOLD + question.upper() + color.END)
    task_data_cosine = generate_task_data_cosine(question, CODER_DICT)
    task_data_jaccard = generate_task_data_jaccard(question, CODER_DICT)
    calculate_score_cosine(question, task_data_cosine)
    calculate_score_jaccard(question, task_data_jaccard)

,videoId,diagnostic,mention,prognostic,motivational_action,motivational_rationale
0,:N2w4spsYm_Q,[none_apply],[embedded],[none_apply],[donation],[celebrity_effect]
1,:eh3bxXHqF2U,[none_apply],[hashtag],[creation],[none_apply],[none_apply]
2,:m7MOlhL6el4,[none_apply],[embedded],[gameplay],[donation],[tree_benefits;celebrity_effect;community_effect]
3,:xygBlTIv9U0,[climate_change;deforestation;other_issues],[dedicated],[spread_the_word;environmental_discussion],[donation;environment_actions],[celebrity_effect;community_effect]
4,:exoFrIJwwV0,[none_apply],[embedded],[gameplay],[donation;support_channel],[celebrity_effect;community_effect]
...,...,...,...,...,...,...
95,:5sp87LcBZLo,[climate_change],[dedicated],[spread_the_word;environmental_discussion],[donation;environment_actions],[tree_benefits;community_effect]
96,:iW4lHAYAO28,[none_apply],[embedded],[knowledge],[donation;more_trees],[celebrity_effect;community_effect]
97,:AIBeJGmGurs,[none_apply],[embedded],[gameplay],[none_apply],[celebrity_effect]
98,:KTrliR_wTUU,[none_apply],[embedded],[donation;gameplay],[none_apply],[none_apply]


DIAGNOSTIC
Fleiss's Kappa using Cosine distance: 0.6263940258623947
Fleiss's Kappa using Jaccard distance: 0.530719803624287
MENTION
Fleiss's Kappa using Cosine distance: 0.5484194681384845
Fleiss's Kappa using Jaccard distance: 0.5642437813956301
PROGNOSTIC
Fleiss's Kappa using Cosine distance: 0.6408164154633531
Fleiss's Kappa using Jaccard distance: 0.6987872569910295
MOTIVATIONAL_ACTION
Fleiss's Kappa using Cosine distance: 0.7149217669619486
Fleiss's Kappa using Jaccard distance: 0.6277675399941245
MOTIVATIONAL_RATIONALE
Fleiss's Kappa using Cosine distance: 0.7617002339826381
Fleiss's Kappa using Jaccard distance: 0.6961087789226704


In [5]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.metrics import agreement,masi_distance,jaccard_distance
from scipy import spatial
import numpy as np


mlb = MultiLabelBinarizer()
arr = mlb.fit_transform([{'none_apply'}, {'climate','forest'} , {'fr'}])
list(mlb.classes_)
names = ["a","b","c"]

a = frozenset([0, 0, 1, 0])
a
# np.array(a)


from nltk.metrics.agreement import AnnotationTask

# cosine similarity is assuming full rank matrices, won't work for rank-deficient
# go through dimension reduction for sparse matrix
def cosine_distance(vec_a,vec_b):
    vec_a, vec_b = list(vec_a), list(vec_b)
    distance = spatial.distance.cosine(vec_a,vec_b)
    return distance

task_data = [('coder1','video0',tuple([1,0,0])),
            ('coder2','video0',tuple([1,0,1])),
            ('coder3','video0',tuple([1,1,0])),
            ('coder1','video1',tuple([1])),
            ('coder2','video1',tuple([1])),
            ('coder3','video1',tuple([1]))]
cosine_task = AnnotationTask(data=task_data,distance = cosine_distance)
cosine_task.multi_kappa()

# a = [1,2,5]
# b = []
# b.append(i for i in a)
# b

(.44*.6)+(.56*.2)

0.376